In [46]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.max_columns = 110

In [47]:
byCategory=pd.read_csv('0614byCategory.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
##將檔期時間調整為至少一個月(檔期開始時間最晚為2019-03-30)
byCategory=byCategory[byCategory.business_hour_order_time_s<'2019-03-30']

In [49]:
##找出檔期長度超過3年,檔期長度即設為1100天
byCategory.loc[byCategory.campagin_period>1100,'campagin_period']=1100

In [50]:
byCategory=byCategory[['business_hour_guid','product_launch_month', 'product_launch_hour',
       'product_launch_weekday','code', 'parent_code', 'is_Presales', 'is_Longtern',  
        'enable_isp','is_Discount','installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 
       'item_orig_price', 'item_price','item_default_daily_amount',
       'quantity_multiplier','shippingdate', 'freight_amount',
        'campagin_period', 'order_deliver_days','unit_price',
       'category_unit_price', 'compare_cate_unit_price',
       'guid_subtotal', 'quantity_sold_guid', 'subtotal_allguid',
       'quantity_sold_allguid', 'avg_unit_price', 'price_gap',
        'hottimes']]


In [51]:
byCategory['unit_price']=byCategory['item_price']/byCategory['quantity_multiplier']


In [52]:
byCategory.columns

Index(['business_hour_guid', 'product_launch_month', 'product_launch_hour',
       'product_launch_weekday', 'code', 'parent_code', 'is_Presales',
       'is_Longtern', 'enable_isp', 'is_Discount', 'installment_3months',
       'installment_6months', 'installment_12months', 'deny_installment',
       'item_orig_price', 'item_price', 'item_default_daily_amount',
       'quantity_multiplier', 'shippingdate', 'freight_amount',
       'campagin_period', 'order_deliver_days', 'unit_price',
       'category_unit_price', 'compare_cate_unit_price', 'guid_subtotal',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'price_gap', 'hottimes'],
      dtype='object')

In [53]:
df=byCategory
df.shape

(224074, 32)

In [54]:
#發現 code,category_unit_price,compare_cate_unit_price 有nan值 因為分類關係
df.code=df.code.fillna(0)
df.shippingdate=df.shippingdate.fillna(0)
df.isna().sum()

business_hour_guid              0
product_launch_month            0
product_launch_hour             0
product_launch_weekday          0
code                            0
parent_code                     0
is_Presales                     0
is_Longtern                     0
enable_isp                      0
is_Discount                     0
installment_3months             0
installment_6months             0
installment_12months            0
deny_installment                0
item_orig_price                 0
item_price                      0
item_default_daily_amount       0
quantity_multiplier             0
shippingdate                    0
freight_amount                  0
campagin_period                 0
order_deliver_days              0
unit_price                      0
category_unit_price          3528
compare_cate_unit_price      3528
guid_subtotal                   0
quantity_sold_guid              0
subtotal_allguid                0
quantity_sold_allguid           0
avg_unit_price

In [55]:
df['category_unit_price']=df['category_unit_price'].fillna(df['unit_price'])

In [56]:
df['compare_cate_unit_price']=df['compare_cate_unit_price'].fillna(1)

## 將檔案順序打散

In [57]:
df=df.sample(frac=1, replace=False, random_state=1)

In [58]:
df=df.reset_index(drop=True)

將資料分為輸入變數與目標變數

## 特徵欄位進行標準化

In [59]:
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0
1,F0AB8354-EB2A-44F6-87D7-F71DAFFF1D28,6,7,3,166.0,166.0,0,0,0,1,0,0,0,0,890.0,239.0,20,1.0,0.0,0.0,13,0,239.0,359.0,0.666,4302.0,18.0,0.0,0.0,239.0,651.0,1
2,D7EE78D5-DE57-4486-881C-1DF8D8B220BC,9,7,2,1604.0,166.0,0,0,0,1,1,0,0,0,799.0,429.0,20,1.0,7.0,0.0,21,0,429.0,319.0,1.345,2574.0,6.0,0.0,0.0,429.0,370.0,0


In [60]:
target=pd.read_csv('target.csv')
code=pd.read_csv('code_time.csv')
target.head(3)

,business_hour_guid,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,00000793-A001-4C8C-9196-3E6971181246,399.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0,399.0,1.0
1,00001911-C3DF-4143-BC25-8AA5FA1A9AF4,1150.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0,1150.0,1.0
2,00001928-AE75-4AA9-8A47-4BDE96D08C2D,23040.0,0.0,0.0,11520.0,0.5,23040.0,1.0,23040.0,1.0,23040.0,1.0


In [61]:
df=pd.merge(df,target,how='left')
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.200,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0
1,F0AB8354-EB2A-44F6-87D7-F71DAFFF1D28,6,7,3,166.0,166.0,0,0,0,1,0,0,0,0,890.0,239.0,20,1.0,0.0,0.0,13,0,239.0,359.0,0.666,4302.0,18.0,0.0,0.0,239.0,651.0,1,4302.0,2390.0,0.556,4302.0,1.000,4302.0,1.0,4302.0,1.0,4302.0,1.0
2,D7EE78D5-DE57-4486-881C-1DF8D8B220BC,9,7,2,1604.0,166.0,0,0,0,1,1,0,0,0,799.0,429.0,20,1.0,7.0,0.0,21,0,429.0,319.0,1.345,2574.0,6.0,0.0,0.0,429.0,370.0,0,2574.0,1287.0,0.500,1287.0,0.500,2574.0,1.0,2574.0,1.0,2574.0,1.0


In [62]:
df=pd.merge(df,code)
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0


In [63]:
parent_code=code
parent_code.columns=['parent_code', 'parent_code_total_sales_amount', 'parent_code_sales_7days', 'parent_code_ratio_7days',
       'parent_code_sales_14days', 'parent_code_ratio_14days', 'parent_code_sales_30days',
       'parent_code_ratio_30days', 'parent_code_sales_60days', 'parent_code_ratio_60days',
       'parent_code_sales_180days', 'parent_code_ratio_180days', 'parent_code_max_7days',
       'parent_code_min_7days', 'parent_code_median_7days', 'parent_code_mean_7days',
       'parent_code_max_14days', 'parent_code_min_14days', 'parent_code_median_14days',
       'parent_code_mean_14days', 'parent_code_max_30days', 'parent_code_min_30days',
       'parent_code_median_30days', 'parent_code_mean_30days', 'parent_code_max_60days',
       'parent_code_min_60days', 'parent_code_median_60days', 'parent_code_mean_60days',
       'parent_code_max_180days', 'parent_code_min_180days', 'parent_code_median_180days',
       'parent_code_mean_180days']
parent_code.head(3)

,parent_code,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,0.0,18506893.0,5865675.0,0.317,8728702.0,0.472,12569867.0,0.679,15237826.0,0.823,17362861.0,0.938,9999.0,100.0,1028.524461,899.0,9999.0,100.0,1043.728566,899.0,16990.0,100.0,1103.200544,950.0,16990.0,100.0,1132.418698,989.0,16990.0,100.0,1140.118261,990.0
1,2.0,1515784.0,684725.0,0.452,970749.0,0.640,1187436.0,0.783,1247623.0,0.823,1382325.0,0.912,4038.0,149.0,541.712816,449.0,8076.0,149.0,555.030875,458.0,8076.0,99.0,560.375649,458.0,8076.0,99.0,561.739307,458.0,8076.0,99.0,569.326606,458.0
2,4.0,44808.0,1851.0,0.041,2631.0,0.059,4509.0,0.101,14144.0,0.316,32940.0,0.735,858.0,490.0,617.000000,503.0,858.0,390.0,526.200000,490.0,939.0,390.0,644.142857,503.0,1878.0,390.0,884.000000,898.5,1878.0,230.0,968.823529,939.0


In [64]:
df=pd.merge(df,parent_code)
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,parent_code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946

## 觀察相關係數

In [65]:
corrtable=(df.corr()[df.corr().index=='sales_7days']).T
sales_7days=corrtable.sort_values(ascending=False,by='sales_7days')
sales_7days=sales_7days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_7days

,sales_7days
price_gap,0.092855
unit_price,0.087693
avg_unit_price,0.087487
item_price,0.081142
item_orig_price,0.078186
compare_cate_unit_price,0.052588
code_max_60days,0.049080
code_max_30days,0.045493
deny_installment,0.039579
code_median_7days,0.039174


In [66]:
corrtable=(df.corr()[df.corr().index=='sales_14days']).T
sales_14days=corrtable.sort_values(ascending=False,by='sales_14days')
sales_14days=sales_14days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_14days

,sales_14days
price_gap,0.100835
avg_unit_price,0.095470
unit_price,0.095354
item_price,0.091266
item_orig_price,0.087832
compare_cate_unit_price,0.057756
code_max_60days,0.053803
code_median_14days,0.049861
code_max_30days,0.049501
code_median_7days,0.049326


In [67]:
corrtable=(df.corr()[df.corr().index=='sales_30days']).T
sales_30days=corrtable.sort_values(ascending=False,by='sales_30days')
sales_30days=sales_30days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_30days

,sales_30days
price_gap,0.106340
avg_unit_price,0.105410
unit_price,0.105327
item_price,0.103596
item_orig_price,0.096455
compare_cate_unit_price,0.065042
code_median_14days,0.060582
code_median_30days,0.059866
code_median_7days,0.059829
code_median_60days,0.059735


In [68]:
corrtable=(df.corr()[df.corr().index=='sales_60days']).T
sales_60days=corrtable.sort_values(ascending=False,by='sales_60days')
sales_60days=sales_60days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_60days

,sales_60days
unit_price,0.114531
avg_unit_price,0.114444
item_price,0.113826
price_gap,0.108478
item_orig_price,0.103314
code_median_14days,0.072308
code_median_30days,0.071681
code_median_60days,0.071558
code_median_7days,0.071362
code_median_180days,0.070914


In [69]:
corrtable=(df.corr()[df.corr().index=='sales_180days']).T
sales_180days=corrtable.sort_values(ascending=False,by='sales_180days')
sales_180days=sales_180days.drop(index=['quantity_sold_guid','ratio_7days','ratio_14days',
                                    'ratio_30days','ratio_60days','ratio_180days',
                                    'guid_subtotal','total_sales_amount','sales_7days',
                                    'sales_14days','sales_30days','sales_60days','sales_180days',
                                    'total_sales_amount'])
sales_180days

,sales_180days
item_price,0.117506
avg_unit_price,0.117474
unit_price,0.117419
price_gap,0.107218
item_orig_price,0.106314
code_median_14days,0.084806
code_median_30days,0.083953
code_median_60days,0.083722
code_median_7days,0.083392
code_median_180days,0.083207


## 因為code的類別數較大,故使用計數編碼

In [70]:
parent_code=df.parent_code.value_counts().to_frame()
parent_code=parent_code.rename(columns={'parent_code':'parent_code_counts'})
parent_code.index.name='parent_code'
#code.columns=[['code','code_counts']]
parent_code=parent_code.reset_index(drop=False)
df=pd.merge(df,parent_code,how='left')
df=df.drop(columns=['parent_code'])

df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,code,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,536.0,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,536.0,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,536.0,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.94

In [71]:
code=df.code.value_counts().to_frame()
code=code.rename(columns={'code':'code_counts'})
code.index.name='code'
#code.columns=[['code','code_counts']]
code=code.reset_index(drop=False)
df=pd.merge(df,code,how='left')

df=df.drop(columns=['code'])
df.head()

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts,code_counts
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.9

In [72]:
df['orig_unit_price']=df['item_orig_price']/df['quantity_multiplier']

In [73]:
df.head(3)

,business_hour_guid,product_launch_month,product_launch_hour,product_launch_weekday,is_Presales,is_Longtern,enable_isp,is_Discount,installment_3months,installment_6months,installment_12months,deny_installment,item_orig_price,item_price,item_default_daily_amount,quantity_multiplier,shippingdate,freight_amount,campagin_period,order_deliver_days,unit_price,category_unit_price,compare_cate_unit_price,guid_subtotal,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,price_gap,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days,code_total_sales_amount,code_sales_7days,code_ratio_7days,code_sales_14days,code_ratio_14days,code_sales_30days,code_ratio_30days,code_sales_60days,code_ratio_60days,code_sales_180days,code_ratio_180days,code_max_7days,code_min_7days,code_median_7days,code_mean_7days,code_max_14days,code_min_14days,code_median_14days,code_mean_14days,code_max_30days,code_min_30days,code_median_30days,code_mean_30days,code_max_60days,code_min_60days,code_median_60days,code_mean_60days,code_max_180days,code_min_180days,code_median_180days,code_mean_180days,parent_code_total_sales_amount,parent_code_sales_7days,parent_code_ratio_7days,parent_code_sales_14days,parent_code_ratio_14days,parent_code_sales_30days,parent_code_ratio_30days,parent_code_sales_60days,parent_code_ratio_60days,parent_code_sales_180days,parent_code_ratio_180days,parent_code_max_7days,parent_code_min_7days,parent_code_median_7days,parent_code_mean_7days,parent_code_max_14days,parent_code_min_14days,parent_code_median_14days,parent_code_mean_14days,parent_code_max_30days,parent_code_min_30days,parent_code_median_30days,parent_code_mean_30days,parent_code_max_60days,parent_code_min_60days,parent_code_median_60days,parent_code_mean_60days,parent_code_max_180days,parent_code_min_180days,parent_code_median_180days,parent_code_mean_180days,parent_code_counts,code_counts,orig_unit_price
0,AB1BA373-E211-4206-A9F1-4A11F4DB5314,7,7,3,0,0,0,1,1,1,1,0,3000.0,1088.0,20,10.0,0.0,0.0,21,0,108.8,299.0,0.364,16320.0,150.0,0.0,0.0,108.8,191.2,0,16320.0,3264.0,0.2,11968.0,0.733,16320.0,1.0,16320.0,1.0,16320.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492,300.0
1,9584B6E4-264C-4FFC-8EF6-5223301E692A,9,7,4,0,1,0,1,1,0,0,0,1380.0,799.0,20,1.0,0.0,0.0,81,0,799.0,299.0,2.672,3995.0,5.0,0.0,0.0,799.0,581.0,0,3995.0,799.0,0.2,799.0,0.200,799.0,0.2,3196.0,0.8,3995.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,52043,28492,1380.0
2,124C1F67-488B-4B55-B449-C45650BDECC7,4,7,1,0,0,0,1,1,1,1,0,1718.0,1150.0,20,1.0,0.0,0.0,21,0,1150.0,299.0,3.846,1150.0,1.0,0.0,0.0,1150.0,568.0,0,1150.0,1150.0,1.0,1150.0,1.000,1150.0,1.0,1150.0,1.0,1150.0,1.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,159588691.0,0.829,182140117.0,0.946,31194.0,11.0,694.68685,569.0,47736.0,10.0,707.835334,579.0,47736.0,10.0,716.509759,593.0,47736.0,10.0,723.213776,594.0,47736.0,10.0,726.450855,589.0,192491020.0,67838255.0,0.352,101498632.0,0.527,140193016.0,0.728,1595

In [74]:
##舊商品新檔次
df_new=df.drop(columns='business_hour_guid')
ohe_df=pd.get_dummies(df_new,columns=['product_launch_month','product_launch_hour','product_launch_weekday'])
subtotal_x=ohe_df.drop(columns=['sales_7days', 'ratio_7days', 'sales_14days', 'ratio_14days',
       'sales_30days', 'ratio_30days', 'sales_60days', 'ratio_60days',
       'sales_180days', 'ratio_180days','guid_subtotal','total_sales_amount'])
subtotal_y=ohe_df[['sales_7days']]


In [75]:
##sales_7days
subtotal_y=df_new[['sales_7days']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subtotal_x, subtotal_y, test_size=0.1, random_state=2)
## 舊商品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(X_train, y_train)
#model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

mean squared error: 50946291.89
mean_absolute_error: 1487.27
r2_score: 0.59


In [ ]:
##ratio_7days
subtotal_y=ohe_df[['ratio_7days']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subtotal_x, subtotal_y, test_size=0.1, random_state=2)
## 舊商品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(X_train, y_train)
#model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

In [76]:
##guid_subtotal
subtotal_y=ohe_df[['guid_subtotal']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subtotal_x, subtotal_y, test_size=0.1, random_state=2)
## 舊商品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(X_train, y_train)
#model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

mean squared error: 16263980.59
mean_absolute_error: 1213.74
r2_score: 0.98


In [ ]:
true7daysales=y_test.reset_index(drop=True)
true7daysales['ratio_7_days']=pred_ratio_7days['ratio_7days']
true7daysales['subtotal']=subtotal['subtotal']



In [ ]:
true7daysales['7days_prediction']=true7daysales['subtotal']*true7daysales['ratio_7_days']
true7daysales.head(4)
print("mean squared error: %.2f"
      % mean_squared_error(true7daysales['subtotal_guid'],true7daysales['7days_prediction']))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(true7daysales['subtotal_guid'],true7daysales['7days_prediction']))

print("r2_score: %.2f"
      % r2_score(true7daysales['subtotal_guid'],true7daysales['7days_prediction']))


In [ ]:
true7daysales['subtotal_guid'].sum()/true7daysales['subtotal_guid'].count()

In [ ]:
pred_ratio_7days=pd.DataFrame(Y_pred)
pred_ratio_7days.columns=['ratio_7days']
pred_ratio_7days.head(3)

In [ ]:
subtotal=pd.DataFrame(Y_pred)
subtotal.columns=['subtotal']
subtotal.head(3)

In [ ]:
##ratio
subtotal_y=df_temp[['ratio_7days']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subtotal_x, subtotal_y, test_size=0.1, random_state=2)
## 舊商品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(X_train, y_train)
#model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

In [ ]:
##subtotal
subtotal_y=df_temp[['subtotal_guid']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subtotal_x, subtotal_y, test_size=0.1, random_state=2)
## 舊商品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(X_train, y_train)
#model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

In [ ]:
#subtotal
y_test

In [ ]:
## 新產品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

In [ ]:
def trainTestValSplit(df):
    # 將資料切分為訓練(80%),測試(10%)和驗證(10%)三份。
    train=df.sample(frac=0.8)
    test=df.drop( train.index )
    val=test.sample(frac=0.5)
    test=test.drop( val.index)
    return train,test,val

def dfXYSplit(df,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetName)
    dfY=df[targetName]
    
    return dfX,dfY
train,test,val=trainTestValSplit(ohe_df)

trainX,trainY=dfXYSplit(train,['sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days'])
testX,testY=dfXYSplit(test,['sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days'])
valX,valY=dfXYSplit(val,['sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days'])

In [ ]:
##先預測前7天之銷量
trainY=trainY['sales_7days']
testY=testY['sales_7days']
valY=valY['sales_7days']

## DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

In [ ]:
valY.head(10)

In [ ]:
valPredY[:10]

### XGBoost

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

### SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
model=SVR(gamma='scale', C=1.0, epsilon=0.2)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

In [ ]:
ohe_df

# 僅將資料分為測試集與驗證集 進行建模

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ohe_x, ohe_y, test_size=0.1, random_state=4)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,y_pred))

## 使用網格搜尋法

In [ ]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
clf= xgb.XGBRegressor()
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)
grid_search = GridSearchCV(clf, param_grid,n_jobs=-1, verbose=1)
grid_result = grid_search.fit(x_train, y_train)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
grid_result.best_params_
clf_bestparam = xgb.XGBRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
print("r2_score: %.2f"
      % r2_score(y_test, y_pred))
